In [47]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor
from sklearn.covariance import EllipticEnvelope
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import SGDRegressor
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [48]:
df=pd.read_csv(r"./preprocessed.csv",index_col=0)
df

,hasImage,has3DModel,hasAdditionalAttributions,marketingStatusSimplifiedCd_Pre-Foreclosure,marketingStatusSimplifiedCd_RecentChange,statusText_Sold,sgapt_Unknown Listed By,city_Charlotte,city_Chicago,city_Denver,...,homeType_LOT,homeType_MANUFACTURED,homeType_MULTI_FAMILY,homeType_SINGLE_FAMILY,homeType_TOWNHOUSE,beds,baths,area,rentZestimate,price
0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,3.0,2.0,1224.0,3999.0,819500.0
1,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,723.0,3634.0,880000.0
2,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,6.0,3.0,3396.0,6000.0,1250000.0
3,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,4.0,3.0,2400.0,4964.0,805000.0
4,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,9.0,3.0,4107.0,8970.0,650000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6201,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,5.0,4.0,2366.0,4999.0,1125000.0
6202,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,3.0,3.0,1802.0,3449.0,465000.0
6203,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,4.0,4.0,2875.0,4749.0,1020000.0
6204,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,4.0,2.5,1260.0,3994.0,550000.0


In [49]:

def scale_selected_columns(x_train, x_test, cols):
    scaler = MinMaxScaler()
    x_train_scaled = x_train.copy()  
    x_test_scaled = x_test.copy()    
    scaler.fit(x_train_scaled[cols])  
    x_train_scaled[cols] = scaler.transform(x_train_scaled[cols])  
    x_test_scaled[cols] = scaler.transform(x_test_scaled[cols])
    return x_train_scaled, x_test_scaled

def log_transform(X, cols):
    X_transformed = X.copy()  
    
    X_transformed[cols] = np.log1p(X_transformed[cols])
        
    return X_transformed

def log_transform_y(y):
    return np.log1p(y.copy())

def remove_outliers(X,y,cols=[], threshold=3):
    z_scores = (X[cols] - X[cols].mean()) / X[cols].std()
    index=(np.abs(z_scores)<threshold).all(axis=1)
    return X[index],y[index]



In [50]:
def perform_linear_regression(df):
    r2_list=[]
    
    for i in range(10):
        x = df.drop(columns=["price"])
        y = df["price"]
        
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)

        model = LinearRegression()
        model.fit(x_train, y_train)

        train_predictions = model.predict(x_train)
        test_predictions = model.predict(x_test)

        train_r2 = r2_score(y_train, train_predictions)
        test_r2 = r2_score(y_test, test_predictions)

        r2_list.append({"train_r2":train_r2,"test_r2":test_r2})

    return r2_list

    

results=perform_linear_regression(df)
results

mean_train_r2 = sum(entry['train_r2'] for entry in results) / len(results)
mean_test_r2 = sum(entry['test_r2'] for entry in results) / len(results)

print("Mean train_r2:", mean_train_r2)
print("Mean test_r2:", mean_test_r2)
results

Mean train_r2: 0.8215168477137613
Mean test_r2: 0.5285087824312887


[{'train_r2': 0.8283811543240924, 'test_r2': 0.3219007485161164},
 {'train_r2': 0.8128712051316874, 'test_r2': 0.6774026988291275},
 {'train_r2': 0.8099055835575415, 'test_r2': 0.700570172435975},
 {'train_r2': 0.8106551365848429, 'test_r2': 0.6192642176536693},
 {'train_r2': 0.8354067169689575, 'test_r2': 0.5648903937049126},
 {'train_r2': 0.8095942477215009, 'test_r2': 0.7082442730051932},
 {'train_r2': 0.8420417742388933, 'test_r2': 0.2052859929179598},
 {'train_r2': 0.819567223984051, 'test_r2': 0.5509709757568853},
 {'train_r2': 0.8153581668158918, 'test_r2': 0.5706454655195694},
 {'train_r2': 0.8313872678101533, 'test_r2': 0.3659128859734795}]

- The model is very overfit as there is a big difference between the r2score of the training data and the test data.
- Let's see if removing outliers makes a difference

In [51]:

def perform_linear_regression_zsor(df):
    r2_list=[]
    
    for i in range(100):
        x = df.drop(columns=["price"])
        y = df["price"]
        
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)

        x_train,y_train=remove_outliers(x_train,y_train,["beds","baths","rentZestimate","area"])

        model = LinearRegression()
        model.fit(x_train, y_train)

        train_predictions = model.predict(x_train)
        test_predictions = model.predict(x_test)

        train_r2 = r2_score(y_train, train_predictions)
        test_r2 = r2_score(y_test, test_predictions)

        r2_list.append({"train_r2":train_r2,"test_r2":test_r2})

    return r2_list

    

results=perform_linear_regression_zsor(df)
results

mean_train_r2 = sum(entry['train_r2'] for entry in results) / len(results)
mean_test_r2 = sum(entry['test_r2'] for entry in results) / len(results)

print("Mean train_r2:", mean_train_r2)
print("Mean test_r2:", mean_test_r2)
results


Mean train_r2: 0.7759773703986008
Mean test_r2: 0.8014381434684539


[{'train_r2': 0.7753797540234137, 'test_r2': 0.8087435418511384},
 {'train_r2': 0.7752521770080076, 'test_r2': 0.8153100235421703},
 {'train_r2': 0.7796733631322396, 'test_r2': 0.7854181087621367},
 {'train_r2': 0.7759183668892913, 'test_r2': 0.7872595077357105},
 {'train_r2': 0.7757856084137978, 'test_r2': 0.583457755784353},
 {'train_r2': 0.7778838418784515, 'test_r2': 0.8397982702436814},
 {'train_r2': 0.792272337835895, 'test_r2': 0.7454024009273823},
 {'train_r2': 0.782592704960802, 'test_r2': 0.7826146380527266},
 {'train_r2': 0.7784279032076962, 'test_r2': 0.7825608093534836},
 {'train_r2': 0.773664525750487, 'test_r2': 0.8258064851753775},
 {'train_r2': 0.7717645306593663, 'test_r2': 0.8423452769773196},
 {'train_r2': 0.7856444936829927, 'test_r2': 0.8347579281204691},
 {'train_r2': 0.7695414527850511, 'test_r2': 0.8864418076411102},
 {'train_r2': 0.773748893604929, 'test_r2': 0.7910203352587012},
 {'train_r2': 0.7832850315622552, 'test_r2': 0.8521986691812733},
 {'train_r2': 0

Already a big improvement! Let's try Min Max scaling

In [52]:

def perform_linear_regression_zsor_and_scaling(df):
    r2_list=[]
    
    for i in range(100):
        x = df.drop(columns=["price"])
        y = df["price"]
        
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)

        x_train,y_train=remove_outliers(x_train,y_train,["beds","baths","rentZestimate","area"])
        
        scaler = MinMaxScaler()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)

        
        model = LinearRegression()
        model.fit(x_train, y_train)
        
        train_predictions = model.predict(x_train)
        test_predictions = model.predict(x_test)

        train_r2 = r2_score(y_train, train_predictions)
        test_r2 = r2_score(y_test, test_predictions)

        r2_list.append({"train_r2":train_r2,"test_r2":test_r2})

    return r2_list

    

results=perform_linear_regression_zsor_and_scaling(df)
results

mean_train_r2 = sum(entry['train_r2'] for entry in results) / len(results)
mean_test_r2 = sum(entry['test_r2'] for entry in results) / len(results)

print("Mean train_r2:", mean_train_r2)
print("Mean test_r2:", mean_test_r2)
results


Mean train_r2: 0.7728848601068046
Mean test_r2: -8.094112512648814e+20


[{'train_r2': 0.7742207398159969, 'test_r2': 0.806419328355898},
 {'train_r2': 0.7752481123365171, 'test_r2': -3.553194159602688e+17},
 {'train_r2': 0.7796454863609512, 'test_r2': -1.0497855545315685e+20},
 {'train_r2': 0.7759007201668493, 'test_r2': -2.8686603469731252e+16},
 {'train_r2': 0.7757809015252914, 'test_r2': 0.5838282123546338},
 {'train_r2': 0.7778825939569645, 'test_r2': 0.839724885831026},
 {'train_r2': 0.7918185292310373, 'test_r2': -2.13233540427199e+22},
 {'train_r2': 0.7823757593174921, 'test_r2': -9.186938530240392e+18},
 {'train_r2': 0.7783689977013787, 'test_r2': 0.7825257657310524},
 {'train_r2': 0.7650352456784548, 'test_r2': 0.8244875937615596},
 {'train_r2': 0.7717291956361538, 'test_r2': 0.842431838136769},
 {'train_r2': 0.785632316827781, 'test_r2': 0.8348171000211143},
 {'train_r2': 0.7683440249201126, 'test_r2': 0.8858204229418787},
 {'train_r2': 0.7737488811705472, 'test_r2': 0.7910202277532372},
 {'train_r2': 0.7830211046313627, 'test_r2': 0.852127822803

Scaling performs horribly our r2 score is below 0!!! Why though everybody says scaling is good?!?!

After lot of careful inspection and experimentation we found good results with SGDRegressor instead of LinearRegression

In [53]:

def perform_linear_regression_zsor_and_scaling(df):
    r2_list=[]
    
    for i in range(100):
        x = df.drop(columns=["price"])
        y = df["price"]
        
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)

        
        x_train,y_train=remove_outliers(x_train,y_train,["beds","baths","rentZestimate","area"])
        
        scaler = MinMaxScaler()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)
        

        model = SGDRegressor()
        model.fit(x_train, y_train)
        
        train_predictions = model.predict(x_train)
        test_predictions = model.predict(x_test)

        train_r2 = r2_score(y_train, train_predictions)
        test_r2 = r2_score(y_test, test_predictions)

        r2_list.append({"train_r2":train_r2,"test_r2":test_r2})

    return r2_list

    

results=perform_linear_regression_zsor_and_scaling(df)
results

mean_train_r2 = sum(entry['train_r2'] for entry in results) / len(results)
mean_test_r2 = sum(entry['test_r2'] for entry in results) / len(results)

print("Mean train_r2:", mean_train_r2)
print("Mean test_r2:", mean_test_r2)
results


Mean train_r2: 0.7731906830924913
Mean test_r2: 0.7957802895342944


[{'train_r2': 0.7722371120787398, 'test_r2': 0.8065661913247109},
 {'train_r2': 0.7733788383418314, 'test_r2': 0.8061431658546729},
 {'train_r2': 0.7771343078624865, 'test_r2': 0.7809534729459897},
 {'train_r2': 0.7730811314060517, 'test_r2': 0.7862644646853417},
 {'train_r2': 0.773691173525472, 'test_r2': 0.5824439055587876},
 {'train_r2': 0.7761378978223191, 'test_r2': 0.8308020986284246},
 {'train_r2': 0.7893361014879081, 'test_r2': 0.746837112550305},
 {'train_r2': 0.7797736092951177, 'test_r2': 0.7829141842278431},
 {'train_r2': 0.7758724689896587, 'test_r2': 0.7890078374054477},
 {'train_r2': 0.77130430577046, 'test_r2': 0.8193781615420943},
 {'train_r2': 0.768299100194626, 'test_r2': 0.8398352076429199},
 {'train_r2': 0.7824465383511474, 'test_r2': 0.82469788520547},
 {'train_r2': 0.7674698153436191, 'test_r2': 0.8795184870108619},
 {'train_r2': 0.7717008041728537, 'test_r2': 0.7841977135089951},
 {'train_r2': 0.7805489737117419, 'test_r2': 0.836888800476294},
 {'train_r2': 0.77

Lets try log transformation

In [54]:

def perform_linear_regression_zsor_and_logtrf(df):
    r2_list=[]
    
    for i in range(100):
        x = df.drop(columns=["price"])
        y = df["price"]
        
        y=log_transform_y(y)
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)

        

        x_train,y_train=remove_outliers(x_train,y_train,["beds","baths","rentZestimate","area"])

        model = LinearRegression()
        model.fit(x_train, y_train)
        
        train_predictions = model.predict(x_train)
        test_predictions = model.predict(x_test)

        train_r2 = r2_score(y_train, train_predictions)
        test_r2 = r2_score(y_test, test_predictions)

        r2_list.append({"train_r2":train_r2,"test_r2":test_r2})

    return r2_list

    

results=perform_linear_regression_zsor_and_logtrf(df)
results

mean_train_r2 = sum(entry['train_r2'] for entry in results) / len(results)
mean_test_r2 = sum(entry['test_r2'] for entry in results) / len(results)

print("Mean train_r2:", mean_train_r2)
print("Mean test_r2:", mean_test_r2)
results


Mean train_r2: 0.627510350324328
Mean test_r2: 0.41345637079529285


[{'train_r2': 0.5898185029576255, 'test_r2': 0.5288786219718933},
 {'train_r2': 0.6112269279898137, 'test_r2': 0.4806570585623954},
 {'train_r2': 0.602050054350872, 'test_r2': 0.6188113734237383},
 {'train_r2': 0.5940387424781451, 'test_r2': 0.656892125395111},
 {'train_r2': 0.6329658310034529, 'test_r2': -0.15339920432317933},
 {'train_r2': 0.6109278451144926, 'test_r2': 0.5048525990419082},
 {'train_r2': 0.639545729834634, 'test_r2': 0.42863228580595725},
 {'train_r2': 0.5879892971682942, 'test_r2': 0.6300690787703549},
 {'train_r2': 0.6135722078467761, 'test_r2': 0.5882195526259543},
 {'train_r2': 0.6143509427242766, 'test_r2': 0.5015122404833329},
 {'train_r2': 0.6019613784269152, 'test_r2': 0.5075444280247312},
 {'train_r2': 0.6315709790861717, 'test_r2': 0.4309339335572383},
 {'train_r2': 0.6443695846850912, 'test_r2': 0.38955891656348685},
 {'train_r2': 0.6292042962374501, 'test_r2': 0.5240277768177832},
 {'train_r2': 0.6576091131465842, 'test_r2': 0.38424588485753375},
 {'train

In [55]:

def perform_linear_regression_zsor_and_logtrf(df):
    r2_list=[]
    
    for i in range(100):
        x = df.drop(columns=["price"])
        y = df["price"]
        
        y=log_transform_y(y)
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)

    
        x_train,y_train=remove_outliers(x_train,y_train,["beds","baths","rentZestimate","area"])
        
        scaler = MinMaxScaler()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)

        model = SGDRegressor()
        model.fit(x_train, y_train)
        
        train_predictions = model.predict(x_train)
        test_predictions = model.predict(x_test)

        train_r2 = r2_score(y_train, train_predictions)
        test_r2 = r2_score(y_test, test_predictions)

        r2_list.append({"train_r2":train_r2,"test_r2":test_r2})

    return r2_list

    

results=perform_linear_regression_zsor_and_logtrf(df)
results

mean_train_r2 = sum(entry['train_r2'] for entry in results) / len(results)
mean_test_r2 = sum(entry['test_r2'] for entry in results) / len(results)

print("Mean train_r2:", mean_train_r2)
print("Mean test_r2:", mean_test_r2)
results


Mean train_r2: 0.5655843746356021
Mean test_r2: 0.4467848714543969


[{'train_r2': 0.526109873064712, 'test_r2': 0.49703957550775213},
 {'train_r2': 0.559456184371667, 'test_r2': 0.4258127815254642},
 {'train_r2': 0.5361546993772561, 'test_r2': 0.6521484714928905},
 {'train_r2': 0.5385948141871169, 'test_r2': 0.5657996936036369},
 {'train_r2': 0.5644170737655838, 'test_r2': 0.16192047392503206},
 {'train_r2': 0.5584858497035052, 'test_r2': 0.5171359438483774},
 {'train_r2': 0.5752270708671199, 'test_r2': 0.4595707802689225},
 {'train_r2': 0.5276588594168226, 'test_r2': 0.6565562540620489},
 {'train_r2': 0.5495859964477037, 'test_r2': 0.5767341741226335},
 {'train_r2': 0.556033920963146, 'test_r2': 0.4904486713256565},
 {'train_r2': 0.5522220540739452, 'test_r2': 0.5524866225712647},
 {'train_r2': 0.5650283414596855, 'test_r2': 0.4970596363207278},
 {'train_r2': 0.5849891321237023, 'test_r2': 0.36411818643575344},
 {'train_r2': 0.5625307321933676, 'test_r2': 0.4761897936741708},
 {'train_r2': 0.5880879349012775, 'test_r2': 0.46144691003378724},
 {'train_

Log transform only seems to make our model worse :(

Some of these weights are unreasonably high, this maybe suggests there could be high multicollinearality between our features

Lets try removing unnecessary features using VIF

In [56]:
X=df.drop(columns=["price"])

X_with_const = add_constant(X)

vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X_with_const.values, i) for i in range(1, X_with_const.shape[1])]

print(vif_data)



c:\Users\vigne\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                                        Feature         VIF
0                                      hasImage    1.810791
1                                    has3DModel    1.041078
2                     hasAdditionalAttributions    3.472165
3   marketingStatusSimplifiedCd_Pre-Foreclosure    1.024043
4      marketingStatusSimplifiedCd_RecentChange    2.955682
5                               statusText_Sold   85.470644
6                       sgapt_Unknown Listed By    1.914193
7                                city_Charlotte   50.549307
8                                  city_Chicago  258.709563
9                                   city_Denver  232.826493
10                            city_Indianapolis  157.670007
11                             city_Los Angeles    4.668515
12                               city_Nashville    8.523990
13                                  city_Others   15.254059
14                            city_Philadelphia         inf
15                                 city_

As we can see here some columns have extremely high VIF, we will remove the columns with vif one by one until there is no column with vif more than 10

In [57]:

def remove_high_vif_variables(X, threshold=10):
    removedCols=[]
    X_with_const = add_constant(X)
    
    vif = pd.Series([variance_inflation_factor(X_with_const.values, i) for i in range(X_with_const.shape[1])], index=X_with_const.columns)
    
    vif = vif.drop("const")
    
    while vif.max() > threshold:
        max_vif_variable = vif.idxmax()
        removedCols.append(max_vif_variable)
        
        X = X.drop(columns=max_vif_variable)
        
        X_with_const = add_constant(X)
        vif = pd.Series([variance_inflation_factor(X_with_const.values, i) for i in range(X_with_const.shape[1])], index=X_with_const.columns)
        
        vif = vif.drop("const")
    
    return X,removedCols


X=df.drop(columns=["price"])
X_no_multicollinearity,removedCols= remove_high_vif_variables(X, threshold=10)

df.drop(columns=removedCols,inplace=True)
X_no_multicollinearity

c:\Users\vigne\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
c:\Users\vigne\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
c:\Users\vigne\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


,hasImage,has3DModel,hasAdditionalAttributions,marketingStatusSimplifiedCd_Pre-Foreclosure,marketingStatusSimplifiedCd_RecentChange,statusText_Sold,sgapt_Unknown Listed By,city_Los Angeles,city_Nashville,city_Phoenix,...,state_WA,homeType_CONDO,homeType_LOT,homeType_MANUFACTURED,homeType_MULTI_FAMILY,homeType_TOWNHOUSE,beds,baths,area,rentZestimate
0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,1224.0,3999.0
1,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,723.0,3634.0
2,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,6.0,3.0,3396.0,6000.0
3,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,4.0,3.0,2400.0,4964.0
4,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,9.0,3.0,4107.0,8970.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6201,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,2366.0,4999.0
6202,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,1802.0,3449.0
6203,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,2875.0,4749.0
6204,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2.5,1260.0,3994.0


In [59]:
X=df.drop(columns=["price"])

X_with_const = add_constant(X)

vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X_with_const.values, i) for i in range(1, X_with_const.shape[1])]

print(vif_data)



                                        Feature       VIF
0                                      hasImage  1.806642
1                                    has3DModel  1.040874
2                     hasAdditionalAttributions  3.444204
3   marketingStatusSimplifiedCd_Pre-Foreclosure  1.022901
4      marketingStatusSimplifiedCd_RecentChange  2.899153
5                               statusText_Sold  1.738442
6                       sgapt_Unknown Listed By  1.910374
7                              city_Los Angeles  1.507241
8                                city_Nashville  3.326646
9                                  city_Phoenix  2.079707
10                               city_San Diego  2.055170
11                           city_San Francisco  2.110637
12                                city_San Jose  3.702165
13                                   city_Tampa  1.265359
14                                     state_CO  2.314713
15                                     state_DC  2.020352
16            

In [86]:

def perform_linear_regression_zsor(df):
    r2_list=[]
    
    for i in range(100):
        x = df.drop(columns=["price"])
        y = df["price"]
        
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)

        x_train,y_train=remove_outliers(x_train,y_train,["beds","baths","rentZestimate","area"])


        model = LinearRegression()
        model.fit(x_train, y_train)

        train_predictions = model.predict(x_train)
        test_predictions = model.predict(x_test)

        train_r2 = r2_score(y_train, train_predictions)
        test_r2 = r2_score(y_test, test_predictions)

        r2_list.append({"train_r2":train_r2,"test_r2":test_r2})

    return r2_list,model

    

results,model=perform_linear_regression_zsor(df)
results

mean_train_r2 = sum(entry['train_r2'] for entry in results) / len(results)
mean_test_r2 = sum(entry['test_r2'] for entry in results) / len(results)

print("Mean train_r2:", mean_train_r2)
print("Mean test_r2:", mean_test_r2)
results


Mean train_r2: 0.7727952362167692
Mean test_r2: 0.7999869154329511


[{'train_r2': 0.772029664901891, 'test_r2': 0.8072697814861213},
 {'train_r2': 0.7725249865830363, 'test_r2': 0.8124108362916929},
 {'train_r2': 0.7760471860906726, 'test_r2': 0.7835334502823998},
 {'train_r2': 0.7731565045443213, 'test_r2': 0.783067065727877},
 {'train_r2': 0.7729936136096156, 'test_r2': 0.5850854941923768},
 {'train_r2': 0.7755032818655205, 'test_r2': 0.8390413025926453},
 {'train_r2': 0.789261985566802, 'test_r2': 0.7415930067198456},
 {'train_r2': 0.7785904227940799, 'test_r2': 0.7830800462980623},
 {'train_r2': 0.7755900598462023, 'test_r2': 0.7776175308753335},
 {'train_r2': 0.7704765385357503, 'test_r2': 0.825252319795701},
 {'train_r2': 0.768284942038252, 'test_r2': 0.8411670822478685},
 {'train_r2': 0.7817324453336414, 'test_r2': 0.8332953147244093},
 {'train_r2': 0.766370594998458, 'test_r2': 0.8852192633670425},
 {'train_r2': 0.7706104973517269, 'test_r2': 0.7866112357061877},
 {'train_r2': 0.7795972813473709, 'test_r2': 0.8517012238587043},
 {'train_r2': 0.

the difference in r2 score is negligible after droping those cols

In [87]:

coefficients = model.coef_
column_names = df.drop(columns='price').columns

coefficients_with_names = list(zip(column_names, coefficients))

sorted_coefficients = sorted(coefficients_with_names, key=lambda x: x[1], reverse=True)

sorted_coefficients_df = pd.DataFrame(sorted_coefficients, columns=['Feature', 'Coefficient'])

sorted_coefficients

[('city_San Jose', 523742.7300303789),
 ('city_San Francisco', 262534.66724789946),
 ('statusText_Sold', 246636.99330273384),
 ('state_WA', 202832.45125670132),
 ('city_San Diego', 175679.39273800724),
 ('hasAdditionalAttributions', 76871.2455271921),
 ('city_Los Angeles', 45418.93915605362),
 ('has3DModel', 40810.414217302015),
 ('baths', 32182.13460872042),
 ('city_Nashville', 29496.13544380288),
 ('hasImage', 26576.76818229022),
 ('state_MA', 16016.302817516193),
 ('sgapt_Unknown Listed By', 7918.8416849449495),
 ('rentZestimate', 225.31251499744394),
 ('area', 31.75781851620559),
 ('city_Tampa', -8523.772624111842),
 ('marketingStatusSimplifiedCd_RecentChange', -14017.669278426201),
 ('homeType_MULTI_FAMILY', -14061.461139354185),
 ('homeType_LOT', -17839.250522746104),
 ('beds', -33566.017739126466),
 ('state_CO', -40919.88126568572),
 ('city_Phoenix', -78883.58968144239),
 ('state_DC', -82116.02255210973),
 ('state_PA', -93297.61719933544),
 ('homeType_MANUFACTURED', -97414.01939

In [88]:
import statsmodels.api as sm

X=df.drop(columns=["price"])
y=df["price"]
X_with_const = sm.add_constant(X)

results = sm.OLS(y, X_with_const).fit()
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.801
Model:                            OLS   Adj. R-squared:                  0.800
Method:                 Least Squares   F-statistic:                     776.0
Date:                Sun, 14 Apr 2024   Prob (F-statistic):               0.00
Time:                        17:02:04   Log-Likelihood:                -91945.
No. Observations:                6206   AIC:                         1.840e+05
Df Residuals:                    6173   BIC:                         1.842e+05
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

In [89]:
coefficients = model.coef_
mse = np.mean((model.predict(X) - y) ** 2)  

# Calculate standard errors
n = len(y)
X_mean = np.mean(X)
standard_errors = np.sqrt(mse / np.sum((X - X_mean) ** 2))

# Calculate t-values
t_values = coefficients / standard_errors

# Print t-values
print("T-values:")
print(t_values)


AttributeError: 'LinearRegression' object has no attribute 'bse'